# BETH Dataset - Unsupervised Anomaly Detection

**Purpose:** Detect anomalies in honeypot system call logs using clustering algorithms trained on normal samples.

## Approach

This notebook implements unsupervised anomaly detection using three clustering methods:
- **K-Means**: Distance to nearest cluster centroid as anomaly score
- **DBSCAN**: Distance to nearest cluster as anomaly score  
- **GMM**: Negative log probability as anomaly score: $-\log(p(x))$

## Anomaly Score Metrics

For each method, we use continuous anomaly scores:
1. **K-Means and DBSCAN**: Distance to nearest cluster centroid
2. **GMM**: $-\log(p(x))$ where $p(x)$ is the probability the model assigns to point $x$

We sweep through continuous thresholds to maximize F1-score for detecting "sus" and "evil" anomalies.

## Evaluation Strategy

A robust evaluation strategy is used to ensure unbiased performance metrics:
- **Train**: Models are trained on "normal" samples from the training set only.
- **Optimize Thresholds**: The decision threshold for each model is optimized on the **validation set** to find the value that maximizes the F1-score.
- **Report Final Metrics**: The final, unbiased performance is reported on the **unseen test set** using the threshold determined from the validation set.
- **Visualize**: Clusters are visualized using PCA to understand the data structure.

This approach mimics a real-world scenario where a model is trained, a decision rule is calibrated on a separate dataset, and performance is then measured on completely new data.

**Author:** Joshua Laubach  
**Date:** November 7, 2025

## 1. Setup and Data Loading

### Feature Engineering Pipeline

Using `load_beth(tfidf=True, raw=False)` which:
1. **Loads and preprocesses** numeric features (scaled system call metadata)
2. **Extracts TF-IDF features** from 'args' column (500 features, min_df=2, max_df=0.5)
3. **Combines automatically** into sparse matrices using `combine_numeric_and_tfidf()`
4. **Returns** combined feature matrices (X) and separate label DataFrames (y)

Returns:
- **X_train, X_val, X_test**: Sparse matrices with numeric + TF-IDF features combined
- **y_train, y_val, y_test**: DataFrames with 'sus' and 'evil' label columns
- **feature_names**: List of TF-IDF feature names for interpretability

This approach captures both:
- **Numeric patterns** from system call metadata (timestamps, IDs, flags)
- **Text patterns** from system call arguments via TF-IDF
- **Memory efficiency** via sparse matrix representation

In [ ]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, silhouette_score
from sklearn.cluster import KMeans, DBSCAN
from sklearn.mixture import GaussianMixture

sys.path.append('../src')
from preprocessing import load_beth
from models_unsupervised import *

import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
print("Libraries imported successfully")

In [ ]:
print("="*80)
print("LOADING BETH DATASET WITH TF-IDF FEATURES")
print("="*80)

# Load preprocessed data with TF-IDF features using the new function
# When tfidf=True, returns 7 values: combined sparse matrices + label DataFrames
X_train, X_val, X_test, y_labels_train, y_labels_val, y_labels_test, tfidf_feature_names = load_beth(
    tfidf=True, 
    raw=False,
    max_features=500,
    min_df=2,
    max_df=0.5
)

print(f"\nDataset loaded successfully!")
print(f"  Combined feature matrices (numeric + TF-IDF):")
print(f"    Training:   {X_train.shape} (sparse)")
print(f"    Validation: {X_val.shape} (sparse)")
print(f"    Test:       {X_test.shape} (sparse)")

print(f"\n  Label DataFrames:")
print(f"    Training:   {y_labels_train.shape} with columns {list(y_labels_train.columns)}")
print(f"    Validation: {y_labels_val.shape}")
print(f"    Test:       {y_labels_test.shape}")

print(f"\n  TF-IDF feature names: {len(tfidf_feature_names)} features")

In [ ]:
# Extract labels from label DataFrames
y_sus_train = y_labels_train['sus'].values
y_evil_train = y_labels_train['evil'].values

y_sus_val = y_labels_val['sus'].values
y_evil_val = y_labels_val['evil'].values

y_sus_test = y_labels_test['sus'].values
y_evil_test = y_labels_test['evil'].values

# X_train, X_val, X_test are already combined sparse matrices (numeric + TF-IDF)
# No need to combine them manually!

print(f"\nFeature matrices ready:")
print(f"  X_train: {X_train.shape} - Combined numeric + TF-IDF (sparse)")
print(f"  X_val:   {X_val.shape} - Combined numeric + TF-IDF (sparse)")
print(f"  X_test:  {X_test.shape} - Combined numeric + TF-IDF (sparse)")

print(f"\nLabel distributions:")
print(f"  Training samples: {X_train.shape[0]:,}")
print(f"    Sus (train):    {y_sus_train.sum():,}")
print(f"    Evil (train):   {y_evil_train.sum():,}")

print(f"\n  Validation samples: {X_val.shape[0]:,}")
print(f"    Normal (val):   {((y_sus_val == 0) & (y_evil_val == 0)).sum():,}")
print(f"    Sus (val):      {y_sus_val.sum():,}")
print(f"    Evil (val):     {y_evil_val.sum():,}")

print(f"\n  Test samples: {X_test.shape[0]:,}")
print(f"    Normal (test):  {((y_sus_test == 0) & (y_evil_test == 0)).sum():,}")
print(f"    Sus (test):     {y_sus_test.sum():,}")
print(f"    Evil (test):    {y_evil_test.sum():,}")

## 2. Hyperparameter Tuning

Optimize hyperparameters for each clustering method using a sample of the training data.

In [ ]:
sample_size = min(10000, X_train.shape[0])
np.random.seed(42)
sample_indices = np.random.choice(X_train.shape[0], sample_size, replace=False)
X_train_sample = X_train[sample_indices]

print(f"Using {sample_size:,} samples for hyperparameter tuning")
print(f"This is {100*sample_size/X_train.shape[0]:.1f}% of the training data")
print(f"Sample shape: {X_train_sample.shape} (sparse matrix)")

### 2.1 K-Means: Elbow Method and Silhouette Analysis

Determine optimal number of clusters using elbow method and silhouette scores.

In [ ]:
# Use the elbow_method function from the module
elbow_results = elbow_method(X_train_sample, k_range=range(2, 16))
best_k = elbow_results['suggested_k_silhouette']

print(f"\n{'='*60}")
print(f"SELECTED: k={best_k} (based on best Silhouette Score)")
print(f"{'='*60}")

### 2.2 DBSCAN: Epsilon and Min Samples Tuning

Tune epsilon and min_samples parameters using grid search on silhouette score and noise ratio.

In [ ]:
# Use the tune_dbscan function from the module
# This will try different epsilon and min_samples values
# Expanding the search range since previous run showed best params at the edges
dbscan_results = tune_dbscan(
    X_train_sample, 
    eps_range=[1.0, 2.0, 3.0, 4.0, 5.0], 
    min_samples_range=[5, 10, 20, 30, 40]
)

print(f"\nDBSCAN tuning complete")
print(f"Best parameters: eps={dbscan_results['best_eps']}, min_samples={dbscan_results['best_min_samples']}")

### 2.3 GMM: Number of Components and Covariance Type

Tune number of components and covariance type using BIC and AIC scores.

In [ ]:
# Use the tune_gmm function from the module
# This will try different numbers of components and covariance types
gmm_results = tune_gmm(X_train_sample, n_components_range=range(2, 11), covariance_types=['full', 'tied', 'diag', 'spherical'])

print(f"\nGMM tuning complete")
print(f"Best parameters: n_components={gmm_results['best_n_components']}, covariance_type={gmm_results['best_covariance_type']}")

## 3. Model Training

Train three unsupervised clustering algorithms on normal samples only using optimized hyperparameters.

In [ ]:
# Train all models using the dedicated function from the module
# This will instantiate and fit KMeans, DBSCAN, and GMM anomaly detectors.
# IMPORTANT: Set scaler='none' because we want to preserve raw feature magnitudes!
# Scaling would compress distances and make anomaly detection less effective.
trained_models = train_all_models(
    X_train,
    contamination=0.05, # A default value, will be re-optimized with thresholds later
    scaler='none',  # Keep raw feature magnitudes for distance calculations
    distance_metric='euclidean'
)

kmeans_model = trained_models['kmeans']
dbscan_model = trained_models['dbscan']
gmm_model = trained_models['gmm']

In [ ]:
# Create models dictionary for easy iteration
models = {
    'kmeans': kmeans_model,
    'dbscan': dbscan_model,
    'gmm': gmm_model
}

print(f"\nModels dictionary created with {len(models)} models")
print(f"Models: {list(models.keys())}")

In [ ]:
# DIAGNOSTIC: Verify scaler settings
print("="*60)
print("DIAGNOSTIC: Checking Model Scaler Settings")
print("="*60)
for name, model in models.items():
    print(f"\n{name}:")
    print(f"  scaler_method: {model.scaler_method}")
    print(f"  scaler_obj: {model.scaler_obj}")
    
# Check a few anomaly scores to see their range
print("\n" + "="*60)
print("DIAGNOSTIC: Sample Anomaly Scores on Test Data")
print("="*60)
for name, model in models.items():
    scores = model.decision_function(X_test[:100])  # First 100 samples
    print(f"\n{name} (first 100 test samples):")
    print(f"  Min: {scores.min():.4f}")
    print(f"  Max: {scores.max():.4f}")
    print(f"  Mean: {scores.mean():.4f}")
    print(f"  Median: {np.median(scores):.4f}")

## 4. Anomaly Score Computation and Threshold Optimization

First, compute anomaly scores on validation set for visualization purposes. Then optimize thresholds on test set (see Section 4.2 for methodology explanation).

In [ ]:
# Compute anomaly scores on validation set
print("Computing anomaly scores on validation set...")

kmeans_scores = kmeans_model.decision_function(X_val)
dbscan_scores = dbscan_model.decision_function(X_val)
gmm_scores = gmm_model.decision_function(X_val)

print(f"\nAnomaly scores computed:")
print(f"  K-Means: {kmeans_scores.shape} (min={kmeans_scores.min():.4f}, max={kmeans_scores.max():.4f})")
print(f"  DBSCAN:  {dbscan_scores.shape} (min={dbscan_scores.min():.4f}, max={dbscan_scores.max():.4f})")
print(f"  GMM:     {gmm_scores.shape} (min={gmm_scores.min():.4f}, max={gmm_scores.max():.4f})")

### 4.2. Threshold Optimization on Validation Set

Now that we have anomaly scores for the validation set, we will use this data to find the optimal decision threshold for each model.

**Methodology:**
- We iterate through a range of potential thresholds.
- For each threshold, we classify samples as "anomalous" if their score is higher than the threshold.
- We calculate the F1-score by comparing these predictions to the true `sus` labels in the validation set.
- The threshold that yields the highest F1-score is selected as the optimal one for that model.

This ensures that our decision rule is tuned independently of the final test data. First, let's compute anomaly scores for the test set, which we will need later for the final evaluation.

In [ ]:
# Get anomaly scores for the TEST set (for threshold optimization)
test_scores = {name: model.decision_function(X_test) for name, model in models.items()}

# Print a sample of the scores to verify
for name, scores in test_scores.items():
    print(f"Test scores for '{name}' (first 5): {scores[:5]}")

Now, we'll iterate through each model's test scores and use our `optimize_threshold_for_target` function to find the best threshold for identifying `sus` attacks.

**Key Insight:** Since higher anomaly scores indicate more anomalous behavior, we search for optimal thresholds in the **upper tail** of the score distribution (90th+ percentile). Samples scoring above this threshold will be classified as anomalies

In [ ]:
optimized_thresholds = {}
val_scores_dict = {name: model.decision_function(X_val) for name, model in models.items()}

for name, scores in val_scores_dict.items():
    # Define search bounds based on the validation set's own distribution.
    # The optimal threshold should be in the upper tail of the distribution.
    lower_bound = np.percentile(scores, 90)   # Start search at the 90th percentile of validation scores
    upper_bound = np.percentile(scores, 100)  # End search at the maximum validation score

    print(f"\n{name}: searching thresholds between {lower_bound:.4f} (val 90th %ile) and {upper_bound:.4f} (val 100th %ile)")

    # Find the best threshold for detecting the 'sus' target on the VALIDATION set
    opt_metrics = optimize_threshold_for_target(
        scores=scores, 
        y_true=y_sus_val,  # Using validation set labels
        model_name=name,
        lower_bound=lower_bound,
        upper_bound=upper_bound,
        metric='f1',
        target_name='sus'
    )
    optimized_thresholds[name] = opt_metrics['threshold']

print("\n" + "="*60)
print("Optimized Thresholds (from Validation Set)")
print("="*60)
print(optimized_thresholds)

In [ ]:
# Get predictions on the test set using the optimized thresholds
# Note: Thresholds were optimized on the validation set. 
# These are unbiased performance metrics on the unseen test set.
test_results = get_predictions_all_models(
    models, 
    X_test, 
    thresholds=optimized_thresholds
)

# Display a sample of the results
for name, results in test_results.items():
    print(f"Test results for '{name}':")
    print(f"  Predictions (first 10): {results['predictions'][:10]}")
    print(f"  Scores (first 5): {results['scores'][:5]}\n")

In [ ]:
# Compute performance metrics for each model on both sus and evil targets
from sklearn.metrics import f1_score, precision_score, recall_score

comparison_data = []

for model_name, threshold in optimized_thresholds.items():
    model = models[model_name]
    
    # Get predictions using the optimized threshold
    preds = model.predict(X_test, threshold=threshold)
    
    # Metrics for 'sus' target
    f1_sus = f1_score(y_sus_test, preds, zero_division=0)
    precision_sus = precision_score(y_sus_test, preds, zero_division=0)
    recall_sus = recall_score(y_sus_test, preds, zero_division=0)
    
    comparison_data.append({
        'Model': model_name,
        'Target': 'sus',
        'F1': f1_sus,
        'Precision': precision_sus,
        'Recall': recall_sus
    })
    
    # Metrics for 'evil' target
    f1_evil = f1_score(y_evil_test, preds, zero_division=0)
    precision_evil = precision_score(y_evil_test, preds, zero_division=0)
    recall_evil = recall_score(y_evil_test, preds, zero_division=0)
    
    comparison_data.append({
        'Model': model_name,
        'Target': 'evil',
        'F1': f1_evil,
        'Precision': precision_evil,
        'Recall': recall_evil
    })

comparison_df = pd.DataFrame(comparison_data)

print("\nUnbiased Performance Metrics on Test Set:")
print("(Thresholds were optimized on the validation set)")
display(comparison_df)

In [ ]:

# Get predictions from DBSCAN using its native noise detection logic
# We call .predict() without a threshold. The model's internal logic will
# classify points as anomalous if their nearest neighbor in the training set was noise.
dbscan_noise_preds = dbscan_model.predict(X_test, threshold=None)

# Calculate metrics for 'sus' target
f1_sus_db = f1_score(y_sus_test, dbscan_noise_preds, zero_division=0)
precision_sus_db = precision_score(y_sus_test, dbscan_noise_preds, zero_division=0)
recall_sus_db = recall_score(y_sus_test, dbscan_noise_preds, zero_division=0)

# Calculate metrics for 'evil' target
f1_evil_db = f1_score(y_evil_test, dbscan_noise_preds, zero_division=0)
precision_evil_db = precision_score(y_evil_test, dbscan_noise_preds, zero_division=0)
recall_evil_db = recall_score(y_evil_test, dbscan_noise_preds, zero_division=0)

# Update the DataFrame
# Find the index for the 'dbscan' model and 'sus' target
sus_index = comparison_df[(comparison_df['Model'] == 'dbscan') & (comparison_df['Target'] == 'sus')].index
comparison_df.loc[sus_index, ['F1', 'Precision', 'Recall']] = [f1_sus_db, precision_sus_db, recall_sus_db]

# Find the index for the 'dbscan' model and 'evil' target
evil_index = comparison_df[(comparison_df['Model'] == 'dbscan') & (comparison_df['Target'] == 'evil')].index
comparison_df.loc[evil_index, ['F1', 'Precision', 'Recall']] = [f1_evil_db, precision_evil_db, recall_evil_db]

print("Updated Performance Metrics for DBSCAN (using noise detection):")
display(comparison_df)


### 5.1. A Better Evaluation for DBSCAN

The threshold optimization method works well for K-Means and GMM, but it's not ideal for DBSCAN. DBSCAN's primary strength is identifying noise points (samples that don't belong to any cluster), which it labels as `-1`. A better way to evaluate DBSCAN is to treat all noise points as anomalies.

Let's calculate the metrics for DBSCAN using this more direct approach.

## 5. Performance Summary

**Methodology Recap:**
- Models were trained on normal samples from the training set only (unsupervised learning).
- Decision thresholds were optimized on the **validation set** to maximize F1-score.
- Final, unbiased metrics were reported on the **unseen test set** using the optimized thresholds.

This robust methodology ensures that the reported performance is a realistic estimate of how the models would perform on new, unseen data.
The performance analysis above provides detailed metrics for detecting both "sus" and "evil" anomalies.

## 6. Cluster Visualization with PCA

Visualize clusters in 2D using PCA with similar color shades for normal samples.

In [ ]:
print("Applying PCA for 2D visualization...")

# Note: After preprocessing, validation set has all samples as 'sus' (scaled values)
# For PCA, we'll fit on a sample of the data instead of filtering by label
print("Note: Validation set contains only anomalous samples after preprocessing.")
print("Fitting PCA on a sample of validation data for visualization...\n")

# Take a random sample for PCA fitting (to save memory)
sample_size = min(10000, X_val.shape[0])
np.random.seed(42)
sample_indices = np.random.choice(X_val.shape[0], sample_size, replace=False)
X_val_sample = X_val[sample_indices]

# Convert sparse to dense for PCA
print(f"Converting {sample_size:,} sample points to dense for PCA...")
X_val_sample_dense = X_val_sample.toarray()

pca = PCA(n_components=2, random_state=42)
pca.fit(X_val_sample_dense)

# Transform all validation data
print(f"Transforming all {X_val.shape[0]:,} validation samples...")
X_val_dense = X_val.toarray()
X_val_pca = pca.transform(X_val_dense)

print(f"\nPCA explained variance ratio: {pca.explained_variance_ratio_}")
print(f"Total variance explained: {pca.explained_variance_ratio_.sum():.4f}")

### 6.1 K-Means Clusters in PCA Space

In [ ]:
cluster_labels = kmeans_model.predict(X_val_dense)

fig, ax = plt.subplots(figsize=(10, 8))

blues = plt.cm.Blues(np.linspace(0.3, 0.9, kmeans_model.n_clusters))

for i in range(kmeans_model.n_clusters):
    mask = cluster_labels == i
    ax.scatter(X_val_pca[mask, 0], X_val_pca[mask, 1], 
               c=[blues[i]], alpha=0.6, s=30, 
               label=f'Cluster {i}', edgecolors='none')

ax.set_xlabel('First Principal Component', fontsize=12)
ax.set_ylabel('Second Principal Component', fontsize=12)
ax.set_title('K-Means Clusters Visualized with PCA', fontsize=14, fontweight='bold')
ax.legend(loc='best', fontsize=9, ncol=2)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\nCluster sizes:")
unique, counts = np.unique(cluster_labels, return_counts=True)
for label, count in zip(unique, counts):
    print(f"  Cluster {label}: {count:,} samples")

### 6.2 DBSCAN Clusters in PCA Space

In [ ]:
# Get DBSCAN cluster labels (not using threshold, showing actual clusters)
dbscan_labels = dbscan_model.predict(X_val_dense)

fig, ax = plt.subplots(figsize=(10, 8))

# Separate noise points (label = -1) from cluster members
noise_mask = dbscan_labels == -1
cluster_mask = dbscan_labels != -1

# Plot cluster members in steelblue
ax.scatter(X_val_pca[cluster_mask, 0], X_val_pca[cluster_mask, 1],
           c='steelblue', alpha=0.6, s=30, label='Clustered Points', edgecolors='none')

# Plot noise/outliers as red X's
ax.scatter(X_val_pca[noise_mask, 0], X_val_pca[noise_mask, 1],
           c='darkred', alpha=0.8, s=40, label='Noise/Outliers', marker='x')

ax.set_xlabel('First Principal Component', fontsize=12)
ax.set_ylabel('Second Principal Component', fontsize=12)
ax.set_title('DBSCAN Clusters Visualized with PCA', fontsize=14, fontweight='bold')
ax.legend(loc='best', fontsize=10)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\nDBSCAN clustering results:")
print(f"  Clustered samples: {cluster_mask.sum():,}")
print(f"  Noise/Outliers: {noise_mask.sum():,}")
print(f"  Number of clusters: {len(set(dbscan_labels[cluster_mask]))}")

### 6.3 GMM Components in PCA Space

In [ ]:
gmm_labels = gmm_model.predict(X_val_dense)

fig, ax = plt.subplots(figsize=(10, 8))

greens = plt.cm.Greens(np.linspace(0.3, 0.9, gmm_model.n_components))

for i in range(gmm_model.n_components):
    mask = gmm_labels == i
    ax.scatter(X_val_pca[mask, 0], X_val_pca[mask, 1],
               c=[greens[i]], alpha=0.6, s=30,
               label=f'Component {i}', edgecolors='none')

ax.set_xlabel('First Principal Component', fontsize=12)
ax.set_ylabel('Second Principal Component', fontsize=12)
ax.set_title('GMM Components Visualized with PCA', fontsize=14, fontweight='bold')
ax.legend(loc='best', fontsize=9)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\nComponent sizes:")
unique, counts = np.unique(gmm_labels, return_counts=True)
for label, count in zip(unique, counts):
    print(f"  Component {label}: {count:,} samples")
print(f"\nComponent weights: {gmm_model.model.weights_}")

## 7. Model Comparison

Compare all three methods using the optimized thresholds.

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(14, 5))

sus_data = comparison_df[comparison_df['Target'] == 'sus']
evil_data = comparison_df[comparison_df['Target'] == 'evil']

x = np.arange(len(sus_data))
width = 0.2

metrics = ['F1', 'Precision', 'Recall']
colors = ['steelblue', 'darkorange', 'green']

for ax, data, target in zip(axs, [sus_data, evil_data], ['sus', 'evil']):
    for i, (metric, color) in enumerate(zip(metrics, colors)):
        offset = width * (i - 1)
        ax.bar(x + offset, data[metric], width, label=metric, color=color)
    
    ax.set_xlabel('Model', fontsize=12)
    ax.set_ylabel('Score', fontsize=12)
    ax.set_title(f'Performance Comparison - {target.upper()}', fontsize=13, fontweight='bold')
    ax.set_xticks(x)
    ax.set_xticklabels(data['Model'])
    ax.legend(fontsize=10)
    ax.set_ylim([0, 1.0])
    ax.grid(True, axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

## 8. Anomaly Score Distributions

Visualize how anomaly scores differ between normal, sus, and evil samples.

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Use test set labels for visualization
normal_mask = (y_sus_test == 0) & (y_evil_test == 0)
sus_mask = (y_sus_test == 1) & (y_evil_test == 0)
evil_mask = (y_evil_test == 1)

# Get test scores for visualization
kmeans_test_scores = kmeans_model.decision_function(X_test)
dbscan_test_scores = dbscan_model.decision_function(X_test)
gmm_test_scores = gmm_model.decision_function(X_test)

for ax, scores, name, model_key in zip(
    axes,
    [kmeans_test_scores, dbscan_test_scores, gmm_test_scores],
    ['K-Means', 'DBSCAN', 'GMM'],
    ['kmeans', 'dbscan', 'gmm']
):
    ax.hist(scores[normal_mask], bins=50, alpha=0.6, label='Normal', color='steelblue', density=True)
    ax.hist(scores[sus_mask], bins=50, alpha=0.6, label='Sus', color='orange', density=True)
    ax.hist(scores[evil_mask], bins=50, alpha=0.6, label='Evil', color='darkred', density=True)
    
    # Use the optimized threshold from our dictionary
    threshold = optimized_thresholds[model_key]
    ax.axvline(x=threshold, color='black', linestyle='--', linewidth=2, label='Threshold (sus)')
    
    ax.set_xlabel('Anomaly Score', fontsize=11)
    ax.set_ylabel('Density', fontsize=11)
    ax.set_title(f'{name} Score Distribution', fontsize=12, fontweight='bold')
    ax.legend(fontsize=9)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 8.1 Validation vs Test Score Comparison

Let's visualize how the validation set scores (used for threshold bounds) compare to the test set scores (used for evaluation). This helps us understand why the validation-based threshold search works so well.

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Get validation scores
val_scores_dict = {
    'kmeans': kmeans_model.decision_function(X_val),
    'dbscan': dbscan_model.decision_function(X_val),
    'gmm': gmm_model.decision_function(X_val)
}

# Get test scores (recreate in case it was overwritten)
test_scores_dict = {
    'kmeans': kmeans_model.decision_function(X_test),
    'dbscan': dbscan_model.decision_function(X_test),
    'gmm': gmm_model.decision_function(X_test)
}

for ax, (name, model_key) in zip(axes, [('K-Means', 'kmeans'), ('DBSCAN', 'dbscan'), ('GMM', 'gmm')]):
    val_scores = val_scores_dict[model_key]
    test_scores_arr = test_scores_dict[model_key]  # Get test scores for this model
    threshold = optimized_thresholds[model_key]
    
    # Plot validation scores (normal samples)
    ax.hist(val_scores, bins=50, alpha=0.5, label='Validation (normal)', color='steelblue', density=True)
    
    # Plot test scores separated by class
    ax.hist(test_scores_arr[normal_mask], bins=50, alpha=0.4, label='Test (normal)', color='lightblue', density=True, linestyle='--')
    ax.hist(test_scores_arr[sus_mask], bins=50, alpha=0.5, label='Test (sus)', color='orange', density=True)
    ax.hist(test_scores_arr[evil_mask], bins=50, alpha=0.5, label='Test (evil)', color='darkred', density=True)
    
    # Mark the threshold search bounds
    lower_bound = np.percentile(val_scores, 90)
    upper_bound = np.percentile(val_scores, 100)
    ax.axvline(x=lower_bound, color='green', linestyle=':', linewidth=2, label=f'Search lower (val 90th)')
    ax.axvline(x=upper_bound, color='purple', linestyle=':', linewidth=2, label=f'Search upper (val max)')
    
    # Mark the optimized threshold
    ax.axvline(x=threshold, color='black', linestyle='--', linewidth=2.5, label=f'Optimal threshold')
    
    ax.set_xlabel('Anomaly Score', fontsize=11)
    ax.set_ylabel('Density', fontsize=11)
    ax.set_title(f'{name}: Val vs Test Scores', fontsize=12, fontweight='bold')
    ax.legend(fontsize=8, loc='upper right')
    ax.grid(True, alpha=0.3)

    # **IMPROVEMENT**: Clip the x-axis for better readability
    # Combine all scores to find a reasonable limit that excludes extreme outliers
    all_scores_for_plot = np.concatenate([val_scores, test_scores_arr])
    # Calculate the 99.9th percentile to serve as a sensible upper limit
    clip_limit = np.percentile(all_scores_for_plot, 99.9)
    
    # Only apply the limit if it's a meaningful value (i.e., > 0)
    if clip_limit > 0:
        # Set x-limit with a 5% padding
        ax.set_xlim(-clip_limit * 0.05, clip_limit * 1.05)
        # Add a text annotation to inform the viewer that the axis is clipped
        ax.text(0.98, 0.98, 'X-axis clipped for readability',
                transform=ax.transAxes, ha='right', va='top',
                fontsize=8, bbox=dict(boxstyle='round,pad=0.3', fc='white', alpha=0.7))

plt.tight_layout()
plt.show()

print("\n" + "="*80)
print("THRESHOLD ANALYSIS")
print("="*80)
for name, threshold in optimized_thresholds.items():
    val_scores = val_scores_dict[name]
    test_scores_arr = test_scores_dict[name]  # Use the dict we created above
    
    lower = np.percentile(val_scores, 90)
    upper = np.percentile(val_scores, 100)
    
    print(f"\n{name.upper()}:")
    print(f"  Search range:      [{lower:.2f}, {upper:.2f}]")
    print(f"  Optimal threshold: {threshold:.2f}")
    
    # Handle cases where upper and lower bounds are the same to avoid division by zero
    if upper > lower:
        position = 100 * (threshold - lower) / (upper - lower)
        print(f"  Position:          {position:.1f}% through search range")
    else:
        print(f"  Position:          N/A (search range has zero width)")

    print(f"  Val 90th %ile:     {np.percentile(val_scores, 90):.2f}")
    print(f"  Val median:        {np.percentile(val_scores, 50):.2f}")
    print(f"  Val max:           {np.percentile(val_scores, 100):.2f}")
    print(f"  Test normal 90th:  {np.percentile(test_scores_arr[normal_mask], 90):.2f}")
    print(f"  Test sus median:   {np.percentile(test_scores_arr[sus_mask], 50):.2f}")
    print(f"  Test evil median:  {np.percentile(test_scores_arr[evil_mask], 50):.2f}")

### 8.2 Box Plot Comparison

A compact view of score distributions showing quartiles, outliers, and threshold placement.

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

for ax, (name, model_key) in zip(axes, [('K-Means', 'kmeans'), ('DBSCAN', 'dbscan'), ('GMM', 'gmm')]):
    val_scores = val_scores_dict[model_key]
    test_scores_arr = test_scores_dict[model_key]
    threshold = optimized_thresholds[model_key]
    
    # Prepare data for box plot
    data_to_plot = [
        val_scores,
        test_scores_arr[normal_mask],
        test_scores_arr[sus_mask],
        test_scores_arr[evil_mask]
    ]
    
    labels = ['Val\n(normal)', 'Test\n(normal)', 'Test\n(sus)', 'Test\n(evil)']
    colors = ['steelblue', 'lightblue', 'orange', 'darkred']
    
    # Create box plot
    bp = ax.boxplot(data_to_plot, labels=labels, patch_artist=True, 
                     showfliers=False,  # Hide outliers for cleaner view
                     widths=0.6)
    
    # Color the boxes
    for patch, color in zip(bp['boxes'], colors):
        patch.set_facecolor(color)
        patch.set_alpha(0.7)
    
    # Add horizontal line for threshold
    ax.axhline(y=threshold, color='black', linestyle='--', linewidth=2, 
               label=f'Threshold: {threshold:.1f}', zorder=10)
    
    # Add shaded region for threshold search bounds
    lower = np.percentile(val_scores, 90)
    upper = np.percentile(val_scores, 100)
    ax.axhspan(lower, upper, alpha=0.1, color='green', label='Search range')
    
    ax.set_ylabel('Anomaly Score', fontsize=11)
    ax.set_title(f'{name} Score Distributions', fontsize=12, fontweight='bold')
    ax.legend(fontsize=9, loc='upper left')
    ax.grid(True, alpha=0.3, axis='y')

    # **IMPROVEMENT**: Clip the y-axis for better readability
    # Find the maximum of the non-outlier range for the first three groups
    # (val_normal, test_normal, test_sus) to set a sensible upper limit.
    # The top whisker of a boxplot is typically Q3 + 1.5 * IQR.
    q3_sus = np.percentile(test_scores_arr[sus_mask], 75)
    q1_sus = np.percentile(test_scores_arr[sus_mask], 25)
    iqr_sus = q3_sus - q1_sus
    
    # Set the upper limit to be slightly above the 'sus' group's upper whisker
    # or the search range, whichever is higher.
    clip_limit = max(q3_sus + 1.5 * iqr_sus, upper)

    if clip_limit > 0:
        # Set y-limit with a 10% padding for better spacing
        ax.set_ylim(-clip_limit * 0.1, clip_limit * 1.1)
        # Add a text annotation
        ax.text(0.98, 0.98, 'Y-axis clipped for readability',
                transform=ax.transAxes, ha='right', va='top',
                fontsize=8, bbox=dict(boxstyle='round,pad=0.3', fc='white', alpha=0.7))

plt.tight_layout()
plt.show()

# Print summary statistics
print("\n" + "="*80)
print("SCORE DISTRIBUTION STATISTICS")
print("="*80)
for name, model_key in [('K-Means', 'kmeans'), ('DBSCAN', 'dbscan'), ('GMM', 'gmm')]:
    val_scores = val_scores_dict[model_key]
    test_scores_arr = test_scores_dict[model_key]  # Use the dict created in previous cell
    threshold = optimized_thresholds[model_key]
    
    print(f"\n{name.upper()}:")
    print(f"  Validation (normal):  median={np.median(val_scores):.1f}, IQR=[{np.percentile(val_scores, 25):.1f}, {np.percentile(val_scores, 75):.1f}]")
    print(f"  Test (normal):        median={np.median(test_scores_arr[normal_mask]):.1f}, IQR=[{np.percentile(test_scores_arr[normal_mask], 25):.1f}, {np.percentile(test_scores_arr[normal_mask], 75):.1f}]")
    print(f"  Test (sus):           median={np.median(test_scores_arr[sus_mask]):.1f}, IQR=[{np.percentile(test_scores_arr[sus_mask], 25):.1f}, {np.percentile(test_scores_arr[sus_mask], 75):.1f}]")
    print(f"  Test (evil):          median={np.median(test_scores_arr[evil_mask]):.1f}, IQR=[{np.percentile(test_scores_arr[evil_mask], 25):.1f}, {np.percentile(test_scores_arr[evil_mask], 75):.1f}]")
    print(f"  Threshold:            {threshold:.1f}")
    
    # Calculate separation only if the median is meaningful
    if np.median(test_scores_arr[sus_mask]) > 0:
        print(f"  Separation (sus):     {(np.median(test_scores_arr[sus_mask]) - threshold):.1f} above threshold")
    if np.median(test_scores_arr[normal_mask]) > 0:
        print(f"  Separation (normal):  {(threshold - np.median(test_scores_arr[normal_mask])):.1f} above normal median")

## 9. Summary and Conclusions

### Complete Analysis Overview

This notebook conducted a comprehensive unsupervised anomaly detection study using the BETH honeypot dataset with three clustering algorithms: K-Means, DBSCAN, and GMM. The analysis followed a robust, multi-stage workflow:

1.  **Training**: Models were trained exclusively on `normal` system call data from the training set.
2.  **Threshold Optimization**: Decision thresholds were optimized on the **validation set** to find the decision rule that best maximized the F1-score for detecting `sus` attacks.
3.  **Performance Reporting**: Final, unbiased metrics were evaluated on the **unseen test set** using the thresholds derived from the validation set.

### Key Findings

1.  **Model Training**: All three models (`KMeans`, `DBSCAN`, `GMM`) were successfully trained on normal data, establishing a baseline of normal behavior.

2.  **Threshold Optimization**: By using the validation set, we were able to find effective decision thresholds for each model without introducing bias from the test set.

3.  **Unbiased Performance Metrics**: The final metrics reported are a realistic measure of how these models would perform on new data, as the test set was held out from both training and threshold tuning.

4.  **Visualization**:
    *   **PCA plots** allowed for 2D visualization of high-dimensional cluster structures.
    *   **Score distribution plots** clearly illustrated the separation between anomaly scores for `normal`, `sus`, and `evil` samples, validating the effectiveness of the models.

### Critical Lessons Learned

1.  **The Importance of a Validation Set**: A separate validation set is crucial for tuning model parameters--including post-processing rules like decision thresholds--without biasing the final evaluation.

2.  **Thresholds are Key**: The performance of an unsupervised anomaly detection system is highly dependent on the choice of decision threshold. A model can be excellent at separating classes, but without a well-chosen threshold, its practical performance can be poor.

3.  **Robust Evaluation is Possible even with Imbalance**: Despite the severe class imbalance, a sound train-validate-test methodology provides a path to generating trustworthy performance metrics.

4.  **Modular Code Design**: By moving core modeling and analysis logic into the `src/models_unsupervised.py` module, the notebook is cleaner, more readable, and focused on orchestration rather than implementation details.

### Final Recommendations

Based on the unbiased performance metrics reported, a recommendation can be made for the best model for this specific task. The model with the best balance of F1-score, precision, and recall for the target anomalies (`sus` and `evil`) should be chosen for deployment. The final comparison table and plots provide all necessary information to make this data-driven decision.
